# Avaliação de Classificadores usando Floresta Aleatória


Floresta Aleatória (_random forest_) é um algoritmo de aprendizagem supervisionada. Este algoritmo cria várias árvores de decisão e as combina para obter uma predição com maior acurácia e estabilidade.

Uma das vantagens do algoritmo de Florestas Aleatória é que este pode ser utilizado tanto para classificação quanto apra regressão. 

Abaixo, é apresentada um modelo simplificado do algoritmo.

<img src="example_alg.png" alt="drawing" width="600"/>

Neste _notebook_ é apresentada uma implementação didática do algoritmo de Floresta Aleatória em _Python_.

No exemplo aqui implementado, será construído um classificador para o _dataset_ Iris.

O _dataset_ Iris é um dos mais (senão o mais) utilizado na literatura de Inteligência Artificial. Ele é composto por 3 classes, e contém 50 exemplos para cada classe (150 exemplos, no total). 

Cada classe corresponde à uma espécie de planta: _Iris Versicolor_, _Iris Virginica_ e _Iris Setosa_.

<img src="irises.png" alt="drawing" width="600"/>

No Iris, cada exemplo é composto por quatro características (além da classe): 
 - comprimento da sépala (_sepal length_)
 - largura da sépala (_sepal width_)
 - comprimento da pétala (_petal length_)
 - largura da pétala (_petal width_)
 
<img src="sepal_vs_petal.png" alt="drawing" width="300"/>

### Funcionamento da Implementação

O primeiro passo para a construção do classificador é importar o _dataset_.

Após, é construída uma classe de árvore de decisão que será usada para implementar a floresta.

Para possibilitar a criação da floresta, os dados então serão subamostrados, e por fim o classificador será 
construído e testado.

### Hiperparâmetros

Abaixo, estará a lista de hiperparâmetros utilizados na implementação junto com uma breve explicação de cada um, de modo que seja fácil testar o classificador com diferentes configurações. 


#### Treinamento
- __Número de Subsets__:
    - __Descrição__: A quantidade de árvores que serão construídas 
    - __Tipo__: Inteiro
    
    
- __Número de Amostras por Subset__: 
    - __Descrição__: a quantidade de amostras utilizadas para construir cada uma das árvores
    - __Tipo__: Inteiro
    
    
- __Método de Subamostragem__:
    - __Descrição__: o método de subamostragem utilizado. Pode ser utilizado _Hold Out_, _Cross Validation_ ou _Bootstrapping_.
    - __Valores__: 
        - _'hold out'_ para _Hold Out_
        - _'cross validation'_ para _Cross Validation_
        - _'bootstrapping'_ para _Bootstrapping_
        
##### _Hold Out_
Os parâmetros aqui definidos só serão utilizados caso o método de subamostragem _Hold Out_ for selecionado.

##### _Cross Validation_
Os parâmetros aqui definidos só serão utilizados caso o método de subamostragem _Cross Validation_ for selecionado.

##### _Bootstrapping_
Os parâmetros aqui definidos só serão utilizados caso o método de subamostragem _Bootstrapping_ for selecionado.    
    
#### Avaliação
- __Métrica de Ganho de Informação__:
    - __Descrição__: a métrica utilizada para avaliação do classificador. Pode ser utilizado a Entropia ou o Índice GINI.
    - __Valores__: 
        - _'entropy'_ para Entropia
        - _'gini'_ para Índice Gini



In [1]:
########### TREINAMENTO #############
# Número de subsets 
num_subsets = 10

# Número de amostras por subset
num_samples = 50

# Método de subamostragem
subsampling_method = 'hold out'


###### Hold Out ######

###### Cross Valdation ######

###### Bootstrapping ######





########### AVALIAÇÃO #############

# Métrica de Ganho de Informação
information_gain = 'entropy'

## Implementação

### 1. Importar o dataset Iris

Para a importação do _dataset_ Iris utiliza-se a extensão _SciKit Learn_, que fornece um conjunto de ferramentas que facilitam o desenvolvimento de aplicações didáticas de Inteligência Artificial. 

Para realizar alterações no _dataset_ (remover linhas, remover colunas, subamostragem, etc.) será utilizada a extensão _Pandas_.

In [2]:
from sklearn import datasets

# Importa o dataset IRIS
iris = datasets.load_iris()

In [3]:
import pandas as pd

# Transforma o Dataset no formato do Pandas de modo a facilitar o acesso aos dados
data = pd.DataFrame({
    'sepal length':iris.data[:,0],
    'sepal width':iris.data[:,1],
    'petal length':iris.data[:,2],
    'petal width':iris.data[:,3],
    'species':iris.target
})

# Mostra as primeiras 5 entradas
data.head()

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### 2. Criar a classe Árvore de Decisão

A classe Árvore de Decisão será utilizada para criar uma Árvore de Decisão à partir de um _dataset_, e classificar um elemento que contenha os mesmos atributos utilizados para sua criação. 

O único hiperparâmetro utilizado é o Ganho de Informação, que será utilizado para escolha da ordem dos atributos para formação da Árvore.

In [66]:
from math import log2, inf
import collections

class DecisionTree:
    """ Cria uma Árvore de Decisão à partir de um Dataset"""
    
    def __init__(self, dataset):
        """ Construtor. 
            dataset: dataset utilizado para treinamento """
        self.dataset=dataset
        
    def informationGain(self, groups, classes):
        """ Calcula o Ganho de Informação de um atributo de um dataset. 
            dataset: dataset para o qual será calculado o ganho de informação.
            feature: atributo para qual será calculado o ganho de informação. """
        
        if information_gain == 'entropy':
            return self.entropy(groups, classes)
            
        elif information_gain == 'gini':
            return self.gini(groups, classes)
        
    def splitDataset(self, index, value, dataset):
        """ Divide um dataset baseado no em um atributo e no valor do atributo 
            index: atributo
            value: valor do atributo
            dataset: dataset """

        left, right = list(), list()

        # Verifica se o índice do atributo escolhido é menor ou igual que o valor de divisão
        # Caso verdadeiro, adiciona em left. Se não, adiciona em right
        for i, row in dataset.iterrows():
            if row[index] <= value:
                left.append(row)
            else:
                right.append(row)
        return left, right

    def getRootNode(self, dataset):
        """ Seleciona o melhor nó raíz de um dataset
            dataset: dataset """

        # Obtem uma lista de classes
        class_values = list(set(row[-1] for row in dataset))

        # Variáveis de apoio: 
        ## b_index: atributo escolhido
        ## b_value: valor de divisão do atributo
        ## b_groups: (left, right) da divisão de index e value
        ## b_score: ganho de informação 
        b_index, b_value, b_groups, b_score = None, None, None, 999

        for index in dataset.columns:
            for i, row in dataset.iterrows():
                groups = self.splitDataset(index, row[index], dataset)
                informationGain = self.informationGain(groups, class_values)
                if informationGain < b_score:
                    b_index, b_value, b_score, b_groups = index, row[index], informationGain, groups


        return {'index':b_index, 'value':b_value, 'groups':b_groups}    
        
    def entropy(self, groups, classes):
        """ Calcula a entropia de um conjunto de elementos
        groups: tupla (left, right), onde left são exemplos positivos e right exemplos negativos
        classes: lista de classes """    

        # Conta o númeto total de exemplos
        n_instances = float(sum([len(group) for group in groups]))

        # Variável de soma do ganho de informação
        entropy = 0.0

        # Faz a soma do índice
        for group in groups:
            size = float(len(group))

            # Se número de amostras for 0, evita divisão por zero
            if size == 0:
                continue

            score = 0.0

            # Avalia o grupo baseado em cada classe
            for class_val in classes:
                p = [row[-1] for row in group].count(class_val) / size

                # Evita log2(0) = Infinito
                if p == 0:
                    continue

                score += p*log2(p)

            # Faz a média
            entropy += (-score) * (size / n_instances)

        return entropy

    def giniIndex(self, groups, classes):
        """ Calcula a Índice Gini de um conjunto de elementos
        groups: tupla (left, right), onde left são exemplos positivos e right exemplos negativos
        classes: lista de classes """    

        # Conta o númeto total de exemplos
        n_instances = float(sum([len(group) for group in groups]))

        # Variável de soma do ganho de informação
        gini = 0.0

        # Faz a soma do índice
        for group in groups:
            size = float(len(group))

            # Se número de amostras for 0, evita divisão por zero
            if size == 0:
                continue

            score = 0.0

            # Avalia o grupo baseado em cada classe
            for class_val in classes:
                p = [row[-1] for row in group].count(class_val) / size            
                score += p * p

            # Faz a média
            gini += (1.0 - score) * (size / n_instances)

        return gini
    
tree = DecisionTree(data)
tree.getRootNode(tree.dataset)


{'index': 'sepal length', 'value': 5.1, 'groups': ([sepal length    5.1
   sepal width     3.5
   petal length    1.4
   petal width     0.2
   species         0.0
   Name: 0, dtype: float64, sepal length    4.9
   sepal width     3.0
   petal length    1.4
   petal width     0.2
   species         0.0
   Name: 1, dtype: float64, sepal length    4.7
   sepal width     3.2
   petal length    1.3
   petal width     0.2
   species         0.0
   Name: 2, dtype: float64, sepal length    4.6
   sepal width     3.1
   petal length    1.5
   petal width     0.2
   species         0.0
   Name: 3, dtype: float64, sepal length    5.0
   sepal width     3.6
   petal length    1.4
   petal width     0.2
   species         0.0
   Name: 4, dtype: float64, sepal length    4.6
   sepal width     3.4
   petal length    1.4
   petal width     0.3
   species         0.0
   Name: 6, dtype: float64, sepal length    5.0
   sepal width     3.4
   petal length    1.5
   petal width     0.2
   species         

### Separar o dataset em Atributos e Classe

In [5]:
# Separar Atributos das Classes
features = data[['sepal length', 'sepal width', 'petal length', 'petal width']]
classes = data['species']

### Bagging

In [6]:



# Cria um conjunto de Subsets vazios
subsets = [[] for i in list(range(num_subsets))]

# Subamostragem aleatória com repetição
for i in range(0, num_subsets):
    subsets[i] = features.sample(n=num_samples, replace=True)
    
# Imprime as primeiras 5 entradas do subset 0
subsets[0].head()   
        

,sepal length,sepal width,petal length,petal width
27,5.2,3.5,1.5,0.2
74,6.4,2.9,4.3,1.3
67,5.8,2.7,4.1,1.0
16,5.4,3.9,1.3,0.4
9,4.9,3.1,1.5,0.1


### Subespaço Aleatório

In [7]:
import math

# Número de Atributos
num_features = math.floor(math.sqrt(features.shape[1]))
columns = features.columns.to_series()

# Seleciona colunas aleatóriamente sem reposição
for i, subset in enumerate(subsets):
    subsets[i] = subset[columns.sample(n=num_features)]
    
# Imprime as primeiras 5 entradas do subset 0
subsets[0].head() 

,petal width,petal length
27,0.2,1.5
74,1.3,4.3
67,1.0,4.1
16,0.4,1.3
9,0.1,1.5


### Construir Árvores